In [1]:
import numpy as np
import torch

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
tokenizer = AutoTokenizer.from_pretrained("models/bert-base-multilingual-cased/")
model = AutoModelForSequenceClassification.from_pretrained("models/bert-base-multilingual-cased/").to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at models/bert-base-multilingual-cased/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from datasets import load_dataset

dataset = load_dataset('multi_eurlex', 'en')

In [7]:
train_dataset = dataset["train"].select(range(5)).map(lambda x: tokenizer(x["text"], padding = "max_length", truncation = True, max_length = 512, return_tensors = "pt"), batched = True).shuffle(seed=42)
validation_dataset = dataset["validation"].select(range(5)).map(lambda x: tokenizer(x["text"], padding = "max_length", truncation = True, max_length = 512, return_tensors = "pt"), batched = True).shuffle(seed=42)

train_dataset = train_dataset.rename_column("labels", "label").remove_columns(["celex_id", "text"])
validation_dataset = validation_dataset.rename_column("labels", "label").remove_columns(["celex_id", "text"])

In [10]:
train_dataset["label"]

[[18, 3, 4, 1], [3, 19, 6], [12, 17, 19, 6], [2, 17], [1, 20, 7, 3, 0]]

In [5]:
from transformers import TrainingArguments, Trainer
import evaluate

training_args = TrainingArguments(output_dir="finetuning_experiment", evaluation_strategy="epoch")
metric = evaluate.load("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = validation_dataset,
    compute_metrics = compute_metrics,
)

trainer.train()

  0%|          | 0/3 [00:00<?, ?it/s]

ValueError: expected sequence of length 4 at dim 1 (got 5)